# Classifying Fronts 2: In code

In [4]:
# imports
from importlib import reload
import os

import numpy as np
import h5py
import xarray
import pandas

from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
import matplotlib.gridspec as gridspec
import seaborn as sns

from skimage import morphology

from wrangler.plotting import cutout
from wrangler.ogcm import llc as wr_llc

from fronts.dbof import utils as dbof_utils
from fronts.dbof import io as dbof_io
from fronts.finding import dev as finding_dev
from fronts.finding import params as finding_params

# DBOF_dev

In [9]:
dbof_dev_file = '../fronts/runs/dbof/dev/llc4320_dbof_dev.json'

# Load up (old files for laptop ease)

In [2]:
cutouts, tbl = finding_dev.load_test_data()

In [5]:
front_params = finding_params.thin_weak_params

# Example

In [3]:
idx = 500
div_sst, sst, sss, Divb2 = finding_dev.parse_idx(cutouts, idx)

## Find the fronts

In [6]:
fronts = finding_dev.algorithms.fronts_from_divb2(Divb2, **front_params)

## Label

In [7]:
flabels = morphology.label(fronts, connectivity=2)

In [9]:
np.unique(flabels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

# Stats

In [40]:
def avg_field(field:np.ndarray, flabels:np.ndarray):
    avgs = []
    Npixs = []
    uni_labels = np.unique(flabels)[1:]
    for ss in uni_labels:
        ipix = flabels == ss
        Npix = np.sum(ipix)
        stat = np.sum(ipix * field) / Npix
        # Save
        avgs.append(stat)
        Npixs.append(Npix)    
    # Return
    return avgs

In [16]:
stats = {}

In [17]:
uni_labels = np.unique(flabels)[1:]
stats['FID'] = uni_labels

## Simple stat (<Divb^2>)

In [42]:
#avg_Divb2 = []
#Npixs = []
#for ss in uni_labels:
#    ipix = flabels == ss
#    Npix = np.sum(ipix)
#    stat = np.sum(ipix * Divb2) / Npix
#    # Save
#    avg_Divb2.append(stat)
#    Npixs.append(Npix)
#
avg_Divb2 = avg_field(Divb2, flabels)
# 
stats['avg_Divb2'] = avg_Divb2

In [19]:
stats['UID'] = [idx]*uni_labels.size

# Length

In [27]:
stats['Npix'] = Npixs

# Lat/lon

In [28]:
# Prepping
R_earth = 6371. # km
circum = 2 * np.pi* R_earth
km_deg = circum / 360.

In [30]:
dx = 2.25 # km/pixel

In [34]:
km_deg

111.19492664455873

## Generate a lat/lon image

In [32]:
lat_0, lon_0 = 32., 110.  # Making these up
cutout_size = 64

In [37]:
np.cos(90.*np.pi/180)

np.float64(6.123233995736766e-17)

In [38]:
lats = lat_0 + np.arange(cutout_size)*dx / km_deg
lat_img = np.outer(lats, np.ones(cutout_size))
# lons (approximate)
lons = lon_0 + np.arange(cutout_size)*dx / (km_deg * np.cos(lat_0*np.pi/180.))
lon_img = np.outer(np.ones(cutout_size), lons)

In [45]:
avg_lats = avg_field(lats, flabels)
avg_lons = avg_field(lons, flabels)

In [46]:
stats['lats'] = avg_lats
stats['lons'] = avg_lons

# Construct a Table

In [47]:
df = pandas.DataFrame(stats)
df

,FID,avg_Divb2,UID,Npix,lats,lons
0,1,2.133320e-14,500,6,32.320383,110.377789
1,2,8.636720e-14,500,68,32.626682,110.738970
2,3,2.856012e-14,500,7,32.283286,110.334045
3,4,2.496077e-14,500,7,32.121408,110.143162
4,5,1.613183e-14,500,16,32.752479,110.887307
5,6,1.999716e-14,500,19,32.431319,110.508602
6,7,1.748580e-14,500,10,32.214488,110.252920
7,8,2.316236e-14,500,9,32.002248,110.002651
8,9,1.843156e-14,500,7,32.870094,111.025996


In [24]:
df2 = pandas.DataFrame(stats)
df3 = pandas.concat([df, df2], ignore_index=True)
df3

,FID,avg_Divb2,UID
0,1,2.133320e-14,500
1,2,8.636720e-14,500
2,3,2.856012e-14,500
3,4,2.496077e-14,500
4,5,1.613183e-14,500
5,6,1.999716e-14,500
6,7,1.748580e-14,500
7,8,2.316236e-14,500
8,9,1.843156e-14,500
9,1,2.133320e-14,500
